# Set Up

In [ ]:
# !pip install cuml

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
import math
import seaborn as sb

In [2]:
# Suppress Warnings for clean notebook
import warnings
warnings.filterwarnings('ignore')

# Data Loading and Exploration

## Loading the Train Dataset

Loading all the datasets in pandas dataframe

In [3]:
df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')

## Exploring the data
### View the First Few Rows of the Dataset:
Use the head() method to display the first few rows of the training dataset. This will give you an initial glimpse of the data's structure.

In [4]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


### Check Basic Dataset Information:
Use the info() method to get an overview of the dataset's columns, data types, and the number of non-null values. This will help you identify missing data.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5237980 entries, 0 to 5237979
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   stock_id                 int64  
 1   date_id                  int64  
 2   seconds_in_bucket        int64  
 3   imbalance_size           float64
 4   imbalance_buy_sell_flag  int64  
 5   reference_price          float64
 6   matched_size             float64
 7   far_price                float64
 8   near_price               float64
 9   bid_price                float64
 10  bid_size                 float64
 11  ask_price                float64
 12  ask_size                 float64
 13  wap                      float64
 14  target                   float64
 15  time_id                  int64  
 16  row_id                   object 
dtypes: float64(11), int64(5), object(1)
memory usage: 679.4+ MB


### Summary Statistics:
Use the describe() method to generate summary statistics for numerical columns. This includes statistics like mean, standard deviation, minimum, maximum, and quartiles.

In [6]:
df.describe()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
count,5.237980e+06,5.237980e+06,5.237980e+06,5.237760e+06,5.237980e+06,5.237760e+06,5.237760e+06,2.343638e+06,2.380800e+06,5.237760e+06,5.237980e+06,5.237760e+06,5.237980e+06,5.237760e+06,5.237892e+06,5.237980e+06
mean,9.928856e+01,2.415100e+02,2.700000e+02,5.715293e+06,-1.189619e-02,9.999955e-01,4.510025e+07,1.001713e+00,9.996601e-01,9.997263e-01,5.181359e+04,1.000264e+00,5.357568e+04,9.999920e-01,-4.756125e-02,1.331005e+04
std,5.787176e+01,1.385319e+02,1.587451e+02,2.051591e+07,8.853374e-01,2.532497e-03,1.398413e+08,7.214705e-01,1.216920e-02,2.499345e-03,1.114214e+05,2.510042e-03,1.293554e+05,2.497509e-03,9.452860e+00,7.619271e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,9.352850e-01,4.316610e+03,7.700000e-05,7.869880e-01,9.349150e-01,0.000000e+00,9.398270e-01,0.000000e+00,9.380080e-01,-3.852898e+02,0.000000e+00
25%,4.900000e+01,1.220000e+02,1.300000e+02,8.453415e+04,-1.000000e+00,9.987630e-01,5.279575e+06,9.963320e-01,9.971000e-01,9.985290e-01,7.374720e+03,9.990290e-01,7.823700e+03,9.987810e-01,-4.559755e+00,6.729000e+03
50%,9.900000e+01,2.420000e+02,2.700000e+02,1.113604e+06,0.000000e+00,9.999670e-01,1.288264e+07,9.998830e-01,9.998890e-01,9.997280e-01,2.196900e+04,1.000207e+00,2.301792e+04,9.999970e-01,-6.020069e-02,1.334500e+04
75%,1.490000e+02,3.610000e+02,4.100000e+02,4.190951e+06,1.000000e+00,1.001174e+00,3.270013e+07,1.003318e+00,1.002590e+00,1.000905e+00,5.583168e+04,1.001414e+00,5.787841e+04,1.001149e+00,4.409552e+00,1.990700e+04
max,1.990000e+02,4.800000e+02,5.400000e+02,2.982028e+09,1.000000e+00,1.077488e+00,7.713682e+09,4.379531e+02,1.309732e+00,1.077488e+00,3.028784e+07,1.077836e+00,5.440500e+07,1.077675e+00,4.460704e+02,2.645400e+04


### Check for Missing Data:
Determine if there are any missing values in the dataset. You can use the isnull() method to identify missing values and the sum() method to count them.

In [7]:
df.isnull().sum()

stock_id                         0
date_id                          0
seconds_in_bucket                0
imbalance_size                 220
imbalance_buy_sell_flag          0
reference_price                220
matched_size                   220
far_price                  2894342
near_price                 2857180
bid_price                      220
bid_size                         0
ask_price                      220
ask_size                         0
wap                            220
target                          88
time_id                          0
row_id                           0
dtype: int64

# Data Preprocessing
## Handling Missing Data
imbalance_size, reference_price, matched_size, far_price, near_price, bid_price, wap, and target has missing values. We'll use mean imputation for the numerical columns and forward fill (FFill) for the target column since it's a time series variable.

In [8]:
numerical_columns = ['imbalance_size', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'ask_price', 'wap', 'target']

# Forward fill for 'target' column
df['target'].fillna(method='ffill', inplace=True)

# Mean imputation for numerical columns
for column in numerical_columns:
    mean_value = df[column].mean()
    df[column].fillna(mean_value, inplace=True)

# Check if there are any remaining missing values
df.isnull().sum()

stock_id                   0
date_id                    0
seconds_in_bucket          0
imbalance_size             0
imbalance_buy_sell_flag    0
reference_price            0
matched_size               0
far_price                  0
near_price                 0
bid_price                  0
bid_size                   0
ask_price                  0
ask_size                   0
wap                        0
target                     0
time_id                    0
row_id                     0
dtype: int64

## Data Encoding
Features Requiring Encoding:

- stock_id (Categorical): This column represents the unique identifier for each stock. We can apply one-hot encoding to convert it into binary columns for each stock.
- imbalance_buy_sell_flag (Categorical): This column represents the direction of auction imbalance and has three categories: 1 (Buy-side imbalance), -1 (Sell-side imbalance), and 0 (No imbalance). We can apply one-hot encoding to this column to convert it into binary columns.

In [9]:
import pandas as pd

# Make a copy of the original DataFrame
df_encoded = df.copy()

# Apply one-hot encoding to 'stock_id' and 'imbalance_buy_sell_flag' with prefixes
df_encoded = pd.get_dummies(df_encoded, columns=['stock_id', 'imbalance_buy_sell_flag'], prefix=['stock', 'imbalance_flag'], drop_first=True)

# Check the current column names in df_encoded
print(df_encoded.columns)

# Drop the original columns if they exist
columns_to_drop = ['stock_id', 'imbalance_buy_sell_flag']
df_encoded.drop(columns=columns_to_drop, errors='ignore', inplace=True)

df_encoded.head()

Index(['date_id', 'seconds_in_bucket', 'imbalance_size', 'reference_price',
       'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size',
       'ask_price',
       ...
       'stock_192', 'stock_193', 'stock_194', 'stock_195', 'stock_196',
       'stock_197', 'stock_198', 'stock_199', 'imbalance_flag_0',
       'imbalance_flag_1'],
      dtype='object', length=216)


,date_id,seconds_in_bucket,imbalance_size,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,...,stock_192,stock_193,stock_194,stock_195,stock_196,stock_197,stock_198,stock_199,imbalance_flag_0,imbalance_flag_1
0,0,0,3180602.69,0.999812,13380276.64,1.001713,0.99966,0.999812,60651.50,1.000026,...,False,False,False,False,False,False,False,False,False,True
1,0,0,166603.91,0.999896,1642214.25,1.001713,0.99966,0.999896,3233.04,1.000660,...,False,False,False,False,False,False,False,False,False,False
2,0,0,302879.87,0.999561,1819368.03,1.001713,0.99966,0.999403,37956.00,1.000298,...,False,False,False,False,False,False,False,False,False,False
3,0,0,11917682.27,1.000171,18389745.62,1.001713,0.99966,0.999999,2324.90,1.000214,...,False,False,False,False,False,False,False,False,False,False
4,0,0,447549.96,0.999532,17860614.95,1.001713,0.99966,0.999394,16485.54,1.000016,...,False,False,False,False,False,False,False,False,False,False


## Feature Engineering
1. Price Spread: Calculated as the difference between the ask price and bid price, capturing the spread between buying and selling prices.

2. Volume-Weighted Average Price (VWAP): Computed separately for each stock, representing the average price at which a stock has traded throughout the day, weighted by volume.

3. Rolling Mean of Matched Size: The 10-period rolling mean of the matched size, measuring trading activity over time for each stock.

4. Imbalance Ratio: The ratio of the imbalance size to the matched size, indicating the degree of order imbalance in the auction.

5. Cumulative Bid Size: The cumulative sum of bid sizes for each stock, showing the total demand at different time points.

6. Skewness of VWAP: A measure of the asymmetry of the VWAP distribution for each stock.

7. Kurtosis of VWAP: A measure of the "tailedness" of the VWAP distribution for each stock, assessing extreme values.

These engineered features provide valuable insights into trading dynamics and aim to enhance the predictive power of the model by capturing various aspects of price and volume behavior for each stock.

In [10]:
# Define a threshold for high volume
threshold = 10000

# Calculate kurtosis of wap for each stock (custom function)
def custom_kurtosis(x):
    return x.kurt()

stock_columns = [col for col in df_encoded.columns if col.startswith('stock_')]

# Calculate price spread
df_encoded['price_spread'] = df_encoded['ask_price'] - df_encoded['bid_price']

# Calculate VWAP for each stock
df_encoded['vwap'] = df_encoded.groupby(stock_columns)['wap'].transform('mean')

# Calculate rolling mean of matched_size for each stock
df_encoded['rolling_mean_matched_size'] = df_encoded.groupby(stock_columns)['matched_size'].transform(lambda x: x.rolling(window=10, min_periods=1).mean())

# Calculate the ratio of imbalance_size to matched_size for each stock
df_encoded['imbalance_ratio'] = df_encoded['imbalance_size'] / df_encoded['matched_size']

# Calculate cumulative sum of bid_size for each stock
df_encoded['cumulative_bid_size'] = df_encoded.groupby(stock_columns)['bid_size'].cumsum()

# Calculate skewness of wap for each stock
df_encoded['wap_skewness'] = df_encoded.groupby(stock_columns)['wap'].transform('skew')

# Calculate kurtosis of wap for each stock
df_encoded['wap_kurtosis'] = df_encoded.groupby(stock_columns)['wap'].transform(custom_kurtosis)

# Create a feature indicating high volume
df_encoded['is_high_volume'] = (df_encoded['bid_size'] + df_encoded['ask_size'] > threshold).astype('bool')

# Display the first few rows of the DataFrame with engineered features
print(df_encoded.head())

   date_id  seconds_in_bucket  imbalance_size  reference_price  matched_size  \
0        0                  0      3180602.69         0.999812   13380276.64   
1        0                  0       166603.91         0.999896    1642214.25   
2        0                  0       302879.87         0.999561    1819368.03   
3        0                  0     11917682.27         1.000171   18389745.62   
4        0                  0       447549.96         0.999532   17860614.95   

   far_price  near_price  bid_price  bid_size  ask_price  ...  \
0   1.001713     0.99966   0.999812  60651.50   1.000026  ...   
1   1.001713     0.99966   0.999896   3233.04   1.000660  ...   
2   1.001713     0.99966   0.999403  37956.00   1.000298  ...   
3   1.001713     0.99966   0.999999   2324.90   1.000214  ...   
4   1.001713     0.99966   0.999394  16485.54   1.000016  ...   

   imbalance_flag_0  imbalance_flag_1  price_spread      vwap  \
0             False              True      0.000214  0.999842  

## Data Scaling
In this dataset, features with varying ranges and magnitudes, such as stock prices and trading volumes, are scaled using the Min-Max Scaling method. This process transforms the data into a common range between 0 and 1, making it more suitable for modeling and ensuring that the model's performance is not affected by differences in feature magnitudes.

In [11]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Define the columns you want to scale
columns_to_scale = [
    'imbalance_size',
    'reference_price',
    'matched_size',
    'far_price',
    'near_price',
    'bid_price',
    'bid_size',
    'ask_price',
    'ask_size',
    'wap',
    'price_spread',
    'vwap',
    'rolling_mean_matched_size',
    'imbalance_ratio',
    'cumulative_bid_size',
    'wap_skewness',
    'wap_kurtosis'
]

# Create a Min-Max Scaler instance
scaler = StandardScaler()

# df_encoded_scaled = df_encoded.copy()
# Apply Min-Max Scaling to the specified columns
df_encoded[columns_to_scale] = scaler.fit_transform(df_encoded[columns_to_scale])

# Display the scaled DataFrame
print(df_encoded.head())

   date_id  seconds_in_bucket  imbalance_size  reference_price  matched_size  \
0        0                  0       -0.123550        -0.072475     -0.226833   
1        0                  0       -0.270464        -0.039305     -0.310773   
2        0                  0       -0.263821        -0.171589     -0.309507   
3        0                  0        0.302327         0.069285     -0.191010   
4        0                  0       -0.256769        -0.183040     -0.194794   

      far_price    near_price  bid_price  bid_size  ask_price  ...  \
0 -2.300533e-15 -1.082577e-13   0.034293  0.079320  -0.095019  ...   
1 -2.300533e-15 -1.082577e-13   0.067903 -0.436007   0.157572  ...   
2 -2.300533e-15 -1.082577e-13  -0.129353 -0.124371   0.013348  ...   
3 -2.300533e-15 -1.082577e-13   0.109115 -0.444158  -0.020118  ...   
4 -2.300533e-15 -1.082577e-13  -0.132954 -0.317067  -0.099003  ...   

   imbalance_flag_0  imbalance_flag_1  price_spread      vwap  \
0             False              

# Feature selection

In [12]:
# Drop unnecessary columns
X = df_encoded.drop(columns=['time_id', 'row_id', 'target'])
X["is_high_volume"] = X["is_high_volume"].astype(int)

y = df_encoded['target']

# Model Training
I am choosing decision tree for now.

In [13]:
# import cupy as cp
# from cuml.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from lightgbm import LGBMRegressor

def evaluate_model(model, y, pred):
    print(f'Mean Square Error (MSE) : ',round(mean_squared_error(y, pred), 1))
    print(f'Mean Absolute Error (MAE) : ', round(mean_absolute_error(y, pred), 1))

# Create the DecisionTreeRegressor model
# model = DecisionTreeRegressor()

In [18]:
x_train, x_val, y_train, y_val = train_test_split(X, y,test_size=0.25,random_state=42, shuffle=True)

## Linear Regression

In [ ]:
lr = LinearRegression()
model = lr.fit(x_train,y_train)

pred_train = model.predict(x_train)
pred_val = model.predict(x_val)

print("Results on training set")
evaluate_model(model, y_train, pred_train)

print("Results on validation set")
evaluate_model(model, y_val, pred_val)

## Decision Tree and Random Forest

In [ ]:
# Load data into a CuPy array
x_train_cp = cp.asarray(x_train)
y_train_cp = cp.asarray(y_train)
x_val_cp = cp.asarray(x_val)
y_val_cp = cp.asarray(y_val)

In [ ]:
dt = DecisionTreeRegressor(criterion="absolute_error", random_state=42)
model = dt.fit(x_train_cp,y_train_cp)

pred_train = model.predict(x_train_cp)
pred_val = model.predict(x_val_cp)

print("Results on training set")
evaluate_model(model, y_train_cp, pred_train)

print("Results on validation set")
evaluate_model(model, y_val_cp, pred_val)

In [ ]:
rf = RandomForestRegressor(n_estimators=100, bootstrap=False, random_state=42)
model = rf.fit(x_train_cp,y_train_cp)

pred_train = model.predict(x_train_cp)
pred_val = model.predict(x_val_cp)

print("Results on training set")
evaluate_model(model, y_train_cp, pred_train)

print("Results on validation set")
evaluate_model(model, y_val_cp, pred_val)

## XGBoost

In [19]:
xgb = XGBRegressor(max_depth=8, n_estimators=4500, objective="reg:squarederror", eval_metric="mae", 
                   tree_method="gpu_hist", device="cuda", reg_lambda=10, learning_rate=0.3, seed=42)
model = xgb.fit(x_train,y_train, eval_set=[(x_train,y_train),(x_val, y_val)], early_stopping_rounds=50, verbose=100)

pred_train = model.predict(x_train)
pred_val = model.predict(x_val)

print("Results on training set")
evaluate_model(model, y_train, pred_train)

print("Results on validation set")
evaluate_model(model, y_val, pred_val)

[04:55:08] WARNING: ../src/learner.cc:767: 
Parameters: { "device" } are not used.

[0]	validation_0-mae:6.36710	validation_1-mae:6.37771
[100]	validation_0-mae:6.13443	validation_1-mae:6.21370
[200]	validation_0-mae:6.02182	validation_1-mae:6.15085
[300]	validation_0-mae:5.93053	validation_1-mae:6.10522
[400]	validation_0-mae:5.84746	validation_1-mae:6.06519
[500]	validation_0-mae:5.76811	validation_1-mae:6.02630
[600]	validation_0-mae:5.69403	validation_1-mae:5.99246
[700]	validation_0-mae:5.62386	validation_1-mae:5.96053
[800]	validation_0-mae:5.55498	validation_1-mae:5.92789
[900]	validation_0-mae:5.49294	validation_1-mae:5.90012
[1000]	validation_0-mae:5.43234	validation_1-mae:5.87445
[1100]	validation_0-mae:5.37652	validation_1-mae:5.85110
[1200]	validation_0-mae:5.31976	validation_1-mae:5.82704
[1300]	validation_0-mae:5.26586	validation_1-mae:5.80471
[1400]	validation_0-mae:5.21349	validation_1-mae:5.78365
[1500]	validation_0-mae:5.16292	validation_1-mae:5.76355
[1600]	validatio

In [19]:
xgb = XGBRegressor(max_depth=8, n_estimators=4500, objective="reg:squarederror", eval_metric="mae", 
                   tree_method="gpu_hist", device="cuda", reg_lambda=10, learning_rate=0.3, seed=42)
model = xgb.fit(x_train,y_train, eval_set=[(x_train,y_train),(x_val, y_val)], early_stopping_rounds=50, verbose=100)

pred_train = model.predict(x_train)
pred_val = model.predict(x_val)

print("Results on training set")
evaluate_model(model, y_train, pred_train)

print("Results on validation set")
evaluate_model(model, y_val, pred_val)

[04:19:33] WARNING: ../src/learner.cc:767: 
Parameters: { "device" } are not used.

[0]	validation_0-mae:6.36710	validation_1-mae:6.37771
[100]	validation_0-mae:6.12983	validation_1-mae:6.20899
[200]	validation_0-mae:6.02148	validation_1-mae:6.15253
[300]	validation_0-mae:5.92558	validation_1-mae:6.10493
[400]	validation_0-mae:5.83946	validation_1-mae:6.06192
[500]	validation_0-mae:5.76005	validation_1-mae:6.02446
[600]	validation_0-mae:5.68790	validation_1-mae:5.99036
[700]	validation_0-mae:5.61895	validation_1-mae:5.95897
[800]	validation_0-mae:5.55610	validation_1-mae:5.93123
[900]	validation_0-mae:5.49503	validation_1-mae:5.90240
[1000]	validation_0-mae:5.43737	validation_1-mae:5.87748
[1100]	validation_0-mae:5.37938	validation_1-mae:5.85238
[1200]	validation_0-mae:5.32190	validation_1-mae:5.82659
[1300]	validation_0-mae:5.26731	validation_1-mae:5.80397
[1400]	validation_0-mae:5.21628	validation_1-mae:5.78413
[1500]	validation_0-mae:5.16532	validation_1-mae:5.76351
[1600]	validatio

## LightGBM

In [1]:
# Create and train the LightGBM model
lgb = LGBMRegressor(objective="regression", max_depth=8, seed=42, n_jobs=-1, num_leaves=200, n_estimators = 5000,
                    reg_alpha=0.8, reg_lambda=10, learning_rate = 0.3)
model = lgb.fit(x_train,y_train, eval_set=[(x_train,y_train),(x_val, y_val)], eval_metric="mae", early_stopping_rounds=50, verbose=100)

# Make predictions
pred_train = model.predict(x_train)
pred_val = model.predict(x_val)

print("Results on training set")
evaluate_model(model, y_train, pred_train)

print("Results on validation set")
evaluate_model(model, y_val, pred_val)

NameError: name 'LGBMRegressor' is not defined

In [14]:
# kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [4]:
# scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_absolute_error', n_jobs=-1)

# # Print the scores
# print("Scores:", scores)
# print("Mean score:", np.mean(scores))

In [ ]:
# model.fit(X,y)

# Auto ML

In [ ]:
%%capture
!pip install -U lightautoml

In [ ]:
# Standard python libraries
import os
import time
import pickle

# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
import torch

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.validation.np_iterators import TimeSeriesIterator

In [ ]:
N_THREADS = 10
N_FOLDS = 10
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 10 * 3600
TARGET_NAME = 'target'

In [ ]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [ ]:
train_data.head()

In [ ]:
task = Task('binary', metric = 'auc')

In [ ]:
roles = {
    'target': 'target',
    'drop': [], 
}

In [ ]:
# 'cv': N_FOLDS
automl = TabularAutoML(
    task = task,
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
    general_params = {'use_algos': [['linear_l2','lgb','lgb_tuned', 'xgb']]},
    #nested_cv_params = {'cv': 5,  'n_folds': 2},
    #selection_params = {'mode': 2, 'feature_group_size': 50, 'select_algos': ['linear_l2', 'gbm']},
    tuning_params = {'max_tuning_time': 20 * 60},
    lgb_params = {'default_params': {'num_trees': 2000, 'learning_rate': 0.005, 'early_stopping_rounds': 500},'freeze_defaults': True},
                  
)

In [ ]:
%%time 

oof_pred = automl.fit_predict(train_data, roles = roles, verbose = 2)

In [ ]:
print(f'TRAIN out-of-fold score: {roc_auc_score(train_data[TARGET_NAME].values, oof_pred.data[:, 0])}')

In [ ]:
test_pred = automl.predict(test_data)
print(f'Prediction for te_data:\n{test_pred}\nShape = {test_pred.shape}')

with open('/kaggle/working/test_pred_LR_LGBM.npy', 'wb') as f:
    np.save(f,test_pred.data[:, 0])

In [ ]:
test_pred.shape[0]

# Final Model

In [16]:
del df
del df_encoded

In [18]:
xgb = XGBRegressor(max_depth=8, n_estimators=4500, objective="reg:squarederror", 
                   tree_method="gpu_hist", n_gpus = -1, device="cuda", reg_lambda=10, learning_rate=0.3, seed=42)
model = xgb.fit(X,y, eval_set=[(X,y)],eval_metric=["mae","rmse"], early_stopping_rounds=50, verbose=100)

[06:21:51] WARNING: ../src/learner.cc:767: 
Parameters: { "device", "n_gpus" } are not used.

[0]	validation_0-mae:6.36958	validation_0-rmse:9.39544
[100]	validation_0-mae:6.13803	validation_0-rmse:8.85333
[200]	validation_0-mae:6.03887	validation_0-rmse:8.63398
[300]	validation_0-mae:5.95140	validation_0-rmse:8.46008
[400]	validation_0-mae:5.87343	validation_0-rmse:8.31030
[500]	validation_0-mae:5.79736	validation_0-rmse:8.17243
[600]	validation_0-mae:5.73209	validation_0-rmse:8.05625
[700]	validation_0-mae:5.67156	validation_0-rmse:7.95128
[800]	validation_0-mae:5.60874	validation_0-rmse:7.84247
[900]	validation_0-mae:5.54953	validation_0-rmse:7.74439
[1000]	validation_0-mae:5.49301	validation_0-rmse:7.65209
[1100]	validation_0-mae:5.44093	validation_0-rmse:7.56786
[1200]	validation_0-mae:5.39068	validation_0-rmse:7.48820
[1300]	validation_0-mae:5.33932	validation_0-rmse:7.40732
[1400]	validation_0-mae:5.29272	validation_0-rmse:7.33474
[1500]	validation_0-mae:5.24747	validation_0-rms

# Preprocess Test Data

In [22]:
def preprocess_test_data(tdf):
    
    # Handling missing values with mean
    columns_for_mean_fill = [
        'seconds_in_bucket',
        'imbalance_size',
        'reference_price',
        'matched_size',
        'far_price',
        'near_price',
        'bid_price',
        'bid_size',
        'ask_price',
        'ask_size',
        'wap'
    ]
    missing_columns = tdf[columns_for_mean_fill].columns[tdf[columns_for_mean_fill].isnull().any()].tolist()
    for column in missing_columns:
        tdf[column].fillna(tdf[column].mean(), inplace=True)
    
    # Handling missing values with 0
    columns_for_zero_fill = [
        'stock_id',
        'date_id',
        'imbalance_buy_sell_flag',
    ]
    missing_columns = tdf[columns_for_zero_fill].columns[tdf[columns_for_zero_fill].isnull().any()].tolist()
    for column in missing_columns:
        tdf[column].fillna(0, inplace=True)
        
    # One hot encoding
    tdf_encoded = tdf.copy()
    tdf_encoded = pd.get_dummies(tdf_encoded, columns=['stock_id', 'imbalance_buy_sell_flag'], prefix=['stock', 'imbalance_flag'], drop_first=True)
    columns_to_drop = ['stock_id', 'imbalance_buy_sell_flag']
    tdf_encoded.drop(columns=columns_to_drop, errors='ignore', inplace=True)
    
    # Feature Engineering
    threshold = 10000
    def custom_kurtosis(x):
        return x.kurt()
    stock_columns = [col for col in tdf_encoded.columns if col.startswith('stock_')]
    tdf_encoded['price_spread'] = tdf_encoded['ask_price'] - tdf_encoded['bid_price']
    tdf_encoded['vwap'] = tdf_encoded.groupby(stock_columns)['wap'].transform('mean')
    tdf_encoded['rolling_mean_matched_size'] = tdf_encoded.groupby(stock_columns)['matched_size'].transform(lambda x: x.rolling(window=10, min_periods=1).mean())
    tdf_encoded['imbalance_ratio'] = tdf_encoded['imbalance_size'] / tdf_encoded['matched_size']
    tdf_encoded['cumulative_bid_size'] = tdf_encoded.groupby(stock_columns)['bid_size'].cumsum()
    tdf_encoded['wap_skewness'] = tdf_encoded.groupby(stock_columns)['wap'].transform('skew')
    tdf_encoded['wap_kurtosis'] = tdf_encoded.groupby(stock_columns)['wap'].transform(custom_kurtosis)
    tdf_encoded['is_high_volume'] = (tdf_encoded['bid_size'] + tdf_encoded['ask_size'] > threshold).astype('bool')
    tdf_encoded["is_high_volume"] = tdf_encoded["is_high_volume"].astype(int)
    tdf_encoded.fillna(0, inplace=True)
    
    # Get the columns of the training data
    training_columns = set(X.columns)

    # Get the columns of the test data
    test_columns = set(tdf_encoded.columns)

    # Identify columns missing in the test data
    missing_columns_in_test = training_columns - test_columns

    # Identify columns missing in the training data
    missing_columns_in_training = test_columns - training_columns

    # Add missing columns to the test data with default values (e.g., 0 for numerical columns)
    for col in missing_columns_in_test:
        tdf_encoded[col] = 0  # You can adjust the default value as needed

    # Remove extra columns from the test data
    for col in missing_columns_in_training:
        tdf_encoded.drop(columns=col, inplace=True)

    # Reorder the columns in the test data to match the order in the training data
    tdf_encoded = tdf_encoded[x.columns]
    
    # Define the columns you want to scale
    columns_to_scale = [
        'imbalance_size',
        'reference_price',
        'matched_size',
        'far_price',
        'near_price',
        'bid_price',
        'bid_size',
        'ask_price',
        'ask_size',
        'wap',
        'price_spread',
        'vwap',
        'rolling_mean_matched_size',
        'imbalance_ratio',
        'cumulative_bid_size',
        'wap_skewness',
        'wap_kurtosis'
    ]

    # Create a Scaler instance
    scaler = StandardScaler()

    # Apply Min-Max Scaling to the specified columns
    tdf_encoded[columns_to_scale] = scaler.fit_transform(tdf_encoded[columns_to_scale])

    
    return tdf_encoded

# Prediction

In [30]:
# Create an environment for testing
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()
counter = 0

Exception: You can only call `make_env()` once.

In [31]:
iter_test

In [32]:
# Iterate over test data
for (test, revealed_targets, sample_prediction) in iter_test:
    test.sort_values(by=['date_id'], inplace=True)
    
    # Preprocess the current batch of test data
    tdf_encoded = preprocess_test_data(test)

    # Calculate predictions for the current batch of test data
    test_predictions = model.predict(tdf_encoded)

    # Continue with the rest of the code for making predictions and submitting
    sample_prediction['target'] = test_predictions
    env.predict(sample_prediction)
    counter += 1

You must call `predict()` successfully before you can continue with `iter_test()`


TypeError: cannot unpack non-iterable NoneType object